In [2]:
try:
    import cv2
except:
    import cv2
import numpy as np
import matplotlib.pyplot as plt
import os    

In [3]:
#0:black 1:white 2:blue 3:red 4:orange 5:green 6:yellow
BLACK = (0,0,0)
WHITE = (255,255,255)
BLUE = (255,0,0)
RED = (0,0,255)
GREEN = (0,255,0)
YELLOW = (0,255,255)
ORANGE = (0,128,255)

In [4]:
def imgshow(image):
    cv2.imshow("image",image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def pltshow(image):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    plt.imshow(image)  
    plt.show()

cwd = os.getcwd()

In [6]:
class cubepre():
    def __init__(self,imagelist):
        self.grid = [None,None,None,None,None,None]
        self.sq = [None,None,None,None,None,None]
        self.face = [None,None,None,None,None,None]
        self.stickers = [None,None,None,None,None,None]
        self.img = [None,None,None,None,None,None]
        width = 800
        scale = imagelist[0].shape[1]/width
        height = int(imagelist[0].shape[0]/scale)
        img_area = height*width
        for k in range(len(imagelist)):
            self.img[k] = cv2.resize(imagelist[k].copy(),(width,height))
            self.face_extract(k)
                       
    def face_extract(self,k):
        temp = self.img[k].copy()
        img_area = temp.shape[0]*temp.shape[1]
        temp = cv2.cvtColor(temp,cv2.COLOR_BGR2HSV)
        temp = cv2.cvtColor(temp,cv2.COLOR_BGR2HSV)
        temp = cv2.GaussianBlur(temp,(5,5),0)        
        h,s,v = cv2.split(temp)
        temp = cv2.Canny(v,25,50)
        temp = cv2.GaussianBlur(temp,(5,5),0)
        temp = cv2.dilate(temp,np.ones([5,5]))
        contours,hierarchy = cv2.findContours(temp,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        rect=[]
        areas = []
        for cnt in contours:
            epsilon = 0.1*cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,epsilon,True)
            area = cv2.contourArea(cnt)
            if len(approx)==4 and area>img_area/400 and area<img_area/20:
                boxwidths = cv2.minAreaRect(cnt)
                rect.append(np.int0(cv2.boxPoints(boxwidths)))
                areas.append(area)
        rect = self.contoursort(rect)
        self.stickers[k] = cv2.drawContours(self.img[k].copy(),rect,-1,YELLOW)
        if len(rect) != 9:
            print("invalid",k,len(rect))            
        self.get_faceval(rect,k)
        
    def get_faceval(self,rect,k):
        pixel_b = np.zeros([3,3])
        pixel_g = np.zeros([3,3])
        pixel_r = np.zeros([3,3])
        
        
                                
        mat = self.colormap([pixel_b,pixel_g,pixel_r])
        self.update(mat,k)        

    def contoursort(self,rect):
        rect = sorted(rect,key=lambda b:b[1][1],reverse=False)
        rect[0:3] = sorted(rect[0:3],key=lambda b:b[1][0],reverse=False)
        rect[3:6] = sorted(rect[3:6],key=lambda b:b[1][0],reverse=False)
        rect[6:9] = sorted(rect[6:9],key=lambda b:b[1][0],reverse=False)
        return rect  
        
    
    def colormap(self,colors):
        mat = np.zeros([3,3])
        B,G,R = colors
        col = [BLACK,WHITE,BLUE,RED,ORANGE,GREEN,YELLOW]
        vals = [0,1,2,3,4,5,6]
        dist = np.zeros([len(col),1])
        for i in range(3):
            for j in range(3):                                
                for m in range(len(col)):
                    c = col[m]
                    dist[m] = abs(B[i][j]-c[0]) + abs(G[i][j]-c[1]) + abs(R[i][j]-c[2])   
                ind = np.argmin(dist)
                mat[i][j] = vals[ind]
        return mat
    
    def update(self,mat,k):
        self.grid[k] = mat
        self.face[k] = np.zeros((3,3,3),np.uint8)
        for i in range(3):
            for j in range(3):
                if mat[i,j] == 1:
                    self.face[k][i,j] = WHITE
                elif mat[i,j] == 2:
                    self.face[k][i,j] = BLUE
                elif mat[i,j] == 3:
                    self.face[k][i,j] = RED
                elif mat[i,j] == 4:
                    self.face[k][i,j] = ORANGE
                elif mat[i,j] == 5:
                     self.face[k][i,j] = GREEN
                elif mat[i,j] == 6:
                     self.face[k][i,j] = YELLOW
                else:
                    pass              

In [7]:
face0 = cv2.imread(cwd+"/images/image1.jpeg",1)
face1 = cv2.imread(cwd+"/images/image2.jpeg",1)
face2 = cv2.imread(cwd+"/images/image3.jpeg",1)
face3 = cv2.imread(cwd+"/images/image4.jpeg",1)
face4 = cv2.imread(cwd+"/images/image5.jpeg",1)
face5 = cv2.imread(cwd+"/images/image6.jpeg",1)
cube = cubepre([face0,face1,face2,face3,face4,face5])

In [68]:
imgshow(cube.stickers[5])

In [126]:
img = cube.img[5].copy()
img1 = cube.img[5].copy()

img_area = img.shape[0]*img.shape[1]
temp = img.copy()
temp = cv2.cvtColor(temp,cv2.COLOR_BGR2HSV)
temp = cv2.GaussianBlur(temp,(5,5),0)        
h,s,v = cv2.split(temp)
temp = cv2.Canny(v,25,50)
temp = cv2.GaussianBlur(temp,(5,5),0)
temp = cv2.dilate(temp,np.ones([5,5]))
contours,hierarchy = cv2.findContours(temp,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
rectangles=[]
areas=[]
for i in range(len(contours)):
    cnt = contours[i]
    epsilon = 0.1*cv2.arcLength(cnt,True)
    approx = cv2.approxPolyDP(cnt,epsilon,True)
    area = cv2.contourArea(cnt)
    if len(approx)==4 and area>900 and area<18000:
        coord = cv2.minAreaRect(cnt)
        areas.append(area)
        rect = (np.int0(cv2.boxPoints(coord)))
        rectangles.append(rect)
        cv2.drawContours(img,[rect],0,YELLOW)
        M = cv2.moments(rect)
        Cx = int((M["m10"] / M["m00"]))
        Cy = int((M["m01"] / M["m00"]))
        cv2.putText(img,str(i),(Cx,Cy),cv2.FONT_HERSHEY_SIMPLEX,0.5,YELLOW,2)
imgshow(img)

rect = rectangles.copy()
#
#for i in range(len(rect)):    
#    pts = rect[i].copy()
#    xy = np.zeros((4, 2), dtype = "float32")
#
#    s = pts.sum(axis = 1)
#    xy[0] = pts[np.argmin(s)]
#    xy[2] = pts[np.argmax(s)]
#    diff = np.diff(pts, axis = 1)
#    xy[1] = pts[np.argmin(diff)]
#    xy[3] = pts[np.argmax(diff)]
    
#    rect[i]=xy.copy()

rect = sorted(rect,key=lambda b:b[1][1],reverse=False)
rect[0:3] = sorted(rect[0:3],key=lambda b:b[1][0],reverse=False)
rect[3:6] = sorted(rect[3:6],key=lambda b:b[1][0],reverse=False)
rect[6:9] = sorted(rect[6:9],key=lambda b:b[1][0],reverse=False)

for i in range(len(rect)):
    M = cv2.moments(rect[i])
    Cx = int((M["m10"] / M["m00"]))
    Cy = int((M["m01"] / M["m00"]))
    cv2.putText(img1,str(i),(Cx,Cy),cv2.FONT_HERSHEY_SIMPLEX,0.5,YELLOW,2)
#cv2.drawContours(img1,rect,-1,YELLOW)
imgshow(img1)

In [95]:
print(len(rectangles))
abc = rectangles[0]
abc[1]

9


array([352, 307])

In [90]:
img2 = cube.img[1].copy()
rect = rectangles.copy()
for i in range(len(rect)):
    M = cv2.moments(rect[i])
    Cx = int((M["m10"] / M["m00"]))
    Cy = int((M["m01"] / M["m00"]))
    cv2.putText(img2,str(i),(Cx,Cy),cv2.FONT_HERSHEY_SIMPLEX,0.5,YELLOW,2)
cv2.drawContours(img2,rect,-1,YELLOW)
imgshow(img2)

In [89]:
img1 = cube.img[0].copy()
rect = rectangles.copy()
rect = sorted(rect,key=lambda b:b[1][1],reverse=False)
rect = sorted(rect,key=lambda b:b[1][0],reverse=False)
for i in range(len(rect)):
    M = cv2.moments(rect[i])
    Cx = int((M["m10"] / M["m00"]))
    Cy = int((M["m01"] / M["m00"]))
    cv2.putText(img1,str(i),(Cx,Cy),cv2.FONT_HERSHEY_SIMPLEX,0.5,YELLOW,2)
cv2.drawContours(img1,rect,-1,YELLOW)
imgshow(img1)

In [18]:
img_area

360000